In [1]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate
import pandas as pd

In [2]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [3]:
# !./setuprds.sh
!./setupdynamo.sh

Creating the DynamoDB table that holds the client floats
{
    "StackId": "arn:aws:cloudformation:us-east-1:123456789:stack/client-float-table/f2e802e9-f6ad-4446-8826-bc8d521ab2ea"
}
Adding in item for core, ZAR wholesale float


In [4]:
!./deploylambdas.sh

Deploying the float lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring serv

Serverless: Checking Stack create progress...
..
Serverless: Stack create finished...
Service Information
service: user-activity-api
stage: local
region: eu-west-1
stack: user-activity-api-local
resources: 12
api keys:
  None
endpoints:
  POST - https://2990973418.execute-api.eu-west-1.amazonaws.com/local/save
functions:
  save: activity-save
  balance: activity-balance
layers:
  None
Serverless Enterprise: Run `serverless login` and deploy again to explore, monitor, secure your serverless project for free.
Deploying the migration lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: 

In [5]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [6]:
# Setup the RDS roles
role_payload = { "type": "CREATE_ROLES", "credentials": {
        "account_api_worker": "pwd_for_account_api",
        "save_tx_api_worker": "pwd_for_transaction_api",
        "float_api_worker": "pwd_for_float_api",
        "auth_api_worker": "pwd_for_auth_api_worker"
    }
}
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps(role_payload))

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '300',
   'access-control-allow-origin': '*',
   'server': 'Werkzeug/0.15.4 Python/3.6.8',
   'date': 'Tue, 09 Jul 2019 09:23:36 GMT'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'Payload': <botocore.response.StreamingBody at 0x7ff96f8f8240>}

In [9]:
# Setup the RDS tables
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps({'type': 'SETUP_TABLES'}))

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '68',
   'access-control-allow-origin': '*',
   'server': 'Werkzeug/0.15.4 Python/3.6.8',
   'date': 'Tue, 09 Jul 2019 09:33:59 GMT'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'Payload': <botocore.response.StreamingBody at 0x7ff96f8f8fd0>}

In [10]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [19]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'defaultFloatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [12]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [13]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='account-create', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [14]:
def generate_saving_transaction(account_id, ref_amount = 100, client_id = 'zar_client_co', float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id,
        'clientId': client_id
    }
    return test_saving_dict

In [15]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='activity-save', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']['amount']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Tx for account %s persisted with new balance %s' % (tx['accountId'], persisted_tx['currentBalance']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [16]:
def run_accrual_for_float(client_id = 'zar_client_co', float_id = 'zar_cash_primary', accrued_amount = 100):
    accrualDict = {
        'clientId': client_id,
        'floatId': float_id,
        'accrualAmount': accrued_amount,
        'currency': 'ZAR',
        'unit': 'HUNDREDTH_CENT',
        'backingEntityIdentifier': 'tx-id-backing'
    }
    lambda_result = local_lambda.invoke(FunctionName='float-accrue', InvocationType='RequestResponse', 
                                               Payload=json.dumps(accrualDict, default=str))
    lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
    return lambda_payload

In [17]:
print_deployed_functions()

Function list:  ['float-accrue', 'account-create', 'activity-save', 'activity-balance', 'db-migration-local-migrate']


In [20]:
accounts = create_number_accounts(number_accounts = 10)

Account with ID b12ffd8d-1f24-4b15-a882-cc06770ab938 persisted at 2019-07-09T09:38:56.939Z
Account with ID 48f44ff7-099a-4a86-889a-5a7a049d327d persisted at 2019-07-09T09:38:57.673Z
Account with ID ffc4d6fb-85c7-4609-b0d0-b0a546a835b5 persisted at 2019-07-09T09:38:58.394Z
Account with ID 2bc842bf-db01-4123-b461-827c7599edc5 persisted at 2019-07-09T09:38:59.041Z
Account with ID 3aa8a75f-bb60-4268-ae84-2a5d0a622071 persisted at 2019-07-09T09:38:59.731Z
Account with ID b2b9a38b-666f-4a18-a4c5-c372dfee9ed5 persisted at 2019-07-09T09:39:00.371Z
Account with ID d7e223b5-7b47-4c95-94d6-05fbf211ca89 persisted at 2019-07-09T09:39:01.012Z
Account with ID 9d0b19e4-7580-4f68-8e29-767de7f97d46 persisted at 2019-07-09T09:39:01.636Z
Account with ID 640add17-bf2f-412a-a938-0a43b403dcbe persisted at 2019-07-09T09:39:02.287Z
Account with ID a6c7efc3-904e-4e67-a841-b5a951f0a4e6 persisted at 2019-07-09T09:39:02.951Z


In [21]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts[:100]]
print(tabulate.tabulate(account_info, headers = 'keys'))

Name                AccountId
------------------  ------------------------------------
Marvin Moore        b12ffd8d-1f24-4b15-a882-cc06770ab938
Jeremiah Phelps     48f44ff7-099a-4a86-889a-5a7a049d327d
Audrey Mcclurg      ffc4d6fb-85c7-4609-b0d0-b0a546a835b5
Kristine Mcdougall  2bc842bf-db01-4123-b461-827c7599edc5
Claude Kelly        3aa8a75f-bb60-4268-ae84-2a5d0a622071
Kenna Humphery      b2b9a38b-666f-4a18-a4c5-c372dfee9ed5
Johnathon Jones     d7e223b5-7b47-4c95-94d6-05fbf211ca89
Bernice Poulin      9d0b19e4-7580-4f68-8e29-767de7f97d46
Anna Myers          640add17-bf2f-412a-a938-0a43b403dcbe
Eddie Transou       a6c7efc3-904e-4e67-a841-b5a951f0a4e6


In [22]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='activity-save', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

Lambda result {'statusCode': 500}


In [ ]:
account_balances = seed_savings_for_accounts(accounts[:100], tx_per_account = 3, base_amount = 100 * 100 * 100)

In [ ]:
def merge_dataframes(accounts, account_balances):
    df = pd.DataFrame(account_balances)
    df = df[df['currentBalance'] == df.groupby('accountId')['currentBalance'].transform('max')]
    df = df[['accountId', 'currentBalance']]
    df['currentBalance'] = pd.to_numeric(df['currentBalance'])
    df['balanceRand'] = df['currentBalance']/10000
    account_name_df = pd.DataFrame(accounts)[['accountId', 'floatId', 'userFamilyName', 'userFirstName']]
    account_name_df['Holder Name'] = account_name_df['userFirstName'] + ' ' + account_name_df['userFamilyName']
    account_name_df.set_index('accountId')
    df.set_index('accountId')
    merged_df = pd.merge(account_name_df, df)[['accountId', 'floatId', 'Holder Name', 'balanceRand']]
    return merged_df

In [ ]:
merged_df = merge_dataframes(accounts, account_balances)

In [ ]:
merged_df

In [ ]:
amount_to_accrue = round(random.random() * 100 * 100 * 100)
print('Instructing accrual of : R', amount_to_accrue / (100 * 100))
accrual_result = run_accrual_for_float(accrued_amount = amount_to_accrue)
allocation_details = json.loads(accrual_result['userAllocationTransactions']['body'])
account_transaction_list = allocation_details['allocationRecords']['accountTxIds']
# print(tabulate.tabulate(account_transaction_list, headers = 'keys'))

In [ ]:
# print(accrual_result)
account_transactions = allocation_details['allocationRecords']['accountTxIds']
number_of_float_transactions = len(allocation_details['allocationRecords']['floatTxIds'])
number_of_account_transactions = len(account_transactions)
print('%d float transactions, %d account transactions' % (number_of_float_transactions, number_of_account_transactions))
sum_of_accounts = sum([transaction['amount'] for transaction in account_transactions])
print('Sum of account additions: ', sum_of_accounts)
bonus_allocation = accrual_result['entityAllocations']['bonusShare']
company_allocation = accrual_result['entityAllocations']['clientShare']
print('Bonus allocation: %d, and client co allocation: %d' % (bonus_allocation, company_allocation))
total_allocations = sum_of_accounts + bonus_allocation + company_allocation
print('Of accrual %d, allocations %d' % (amount_to_accrue, total_allocations))
rounding_allocated_to_bonus = isinstance(allocation_details['bonusAllocation'], list)
print('Was there a rounding allocation? : ', rounding_allocated_to_bonus)
print('Rounding allocation record: ', allocation_details['bonusAllocation'])

In [ ]:
balance_payload = { 
    'accountId': accounts[0]['accountId'], 
    'clientId': 'zar_client_co', 
    'floatId': 'zar_cash_primary',
    'currency': 'ZAR',
    'timezone': 'Africa/Johannesburg'
}
balance_projection = local_lambda.invoke(FunctionName='activity-balance', Payload=json.dumps(balance_payload))
projection_decoded = decode_lambda_result(balance_projection)
print(projection_decoded)

In [ ]:
def fetch_account_balance(accountId, clientId = 'zar_client_co', floatId = 'zar_cash_primary', currency = 'ZAR', timezone = 'Africa/Johannesburg'):
    balance_payload = { 
        'accountId': accountId, 
        'clientId': 'zar_client_co', 
        'floatId': 'zar_cash_primary',
        'currency': 'ZAR',
        'timezone': 'Africa/Johannesburg'
    }
    balance_projection = local_lambda.invoke(FunctionName='activity-balance', Payload=json.dumps(balance_payload))
    projection_decoded = decode_lambda_result(balance_projection)
    return projection_decoded

In [ ]:
def fetch_account_balances(accounts):
#     balance_dicts = [fetch_account_balance(account['accountId']) for account in accounts]
    balance_dicts = []
    for account in accounts:
#         print('account: ', account)
        balance_dict = fetch_account_balance(account['accountId'])
        balance_dict['accountId'] = account['accountId']
        projected_dict = {
            'accountId': balance_dict['accountId'],
            'currentBalanceZAR': balance_dict['currentBalance']['amount'] / 10000,
            'balanceEndOfDayZAR': balance_dict['balanceEndOfToday']['amount'] / 10000,
            'balanceNextDayZAR': balance_dict['balanceSubsequentDays'][0]['amount'] / 10000
        }
        balance_dicts.append(projected_dict)
    
    return balance_dicts

In [ ]:
balances_projections = fetch_account_balances(accounts)

In [ ]:
balance_df = pd.DataFrame(balances_projections)
balance_df = balance_df[['accountId', 'currentBalanceZAR', 'balanceEndOfDayZAR', 'balanceNextDayZAR']]
balance_df